In [32]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
# contagem de aging
from collections import Counter

from IPython.display import display, HTML

## modelo
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve
from sklearn.metrics import average_precision_score, mean_absolute_error, roc_curve, auc, roc_auc_score
from scikitplot.metrics import plot_roc
from sklearn.model_selection import cross_val_score

# check xgboost version
from xgboost import XGBClassifier, XGBRegressor

import dill as pickle

## alertas
import warnings
warnings.filterwarnings("ignore")

***
### Funções

In [33]:
def aucur(y_test, preds):
    # calculate AUC
    aucur = roc_auc_score(y_test, preds)
    print('AUC: %.3f' % aucur)

    # calculate roc curve
    fpr, tpr, thresholds = roc_curve(y_test, preds)

    # plot no skill
    plt.plot([0, 1], [0, 1], linestyle='--')
    # plot the roc curve for the model
    plt.plot(fpr, tpr, marker='.')
    # show the plot
    plt.show()

In [34]:
def just_check_nan(df):
    # Check NaN Pandas and Numpy
    missing_val_count_by_column = (df.isna().sum())

    columns_with_nan = missing_val_count_by_column[missing_val_count_by_column > 0]
    print('Dados com NaN:')
    print(columns_with_nan)
    print('')
    print('columns_with_nan.shape', columns_with_nan.shape)
    print('')

In [35]:
def plot_feature_freq(df, feature):
    labels, counts = np.unique(df[feature].dropna(), return_counts=True)
    total = counts[0] + counts[1]
    plt.figure(figsize=(10, 7))
    ax = sns.barplot(x=labels, y=(counts/total)*100, data = df)
    ax2 = sns.barplot(x=labels, y=counts, data = df)

    for p in ax.patches:
        ax.annotate('{:.2f}%'.format(p.get_height()), (p.get_x() + 0.3, p.get_height()+0.6), rotation = 0)

#     ax.set(xlabel='Trocou', ylabel='Porcentagem')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    plt.show()

In [36]:
def plot_feature_freq_mes(df, feature):
    labels, counts = np.unique(df[feature].dropna(), return_counts=True)
    total = counts[0] + counts[1] + counts[2] + counts[3] + counts[4] + counts[5] + counts[6]
    plt.figure(figsize=(10, 7))
    ax = sns.barplot(x=labels, y=(counts/total)*100, data = df)
    ax2 = sns.barplot(x=labels, y=counts, data = df)

    for p in ax.patches:
        ax.annotate('{:.2f}%'.format(p.get_height()), (p.get_x() + 0.3, p.get_height()+0.6), rotation = 0)

#     ax.set(xlabel='Trocou', ylabel='Porcentagem')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    plt.show()

***
## Coleta de dados

In [37]:
FILE = "/kaggle/input/datasetsnew/ta_predicted.csv"

In [38]:
client_chunks = []
dados_teste = pd.read_csv(FILE, sep = ",", 
                          header = 0, 
                          index_col = False, 
                          names = None, 
                          low_memory = False, 
                          chunksize = 10**5) 

In [39]:
for chunk in dados_teste:

    new_clients = pd.DataFrame(chunk)

    # filter data chunk with frac percentage
#     new_clients = new_clients.sample(frac = 0.5)
    new_clients = new_clients.drop_duplicates()

    # saves to list
    client_chunks.append(new_clients)

dados = pd.concat(client_chunks).reset_index(drop = True)

In [40]:
dados = dados.drop(['Unnamed: 0'], axis=1)
dados

,msisdn,imsi,tac,dt_inicio,dt_fim,cat_aparelho,fabricante,modelo,imei,ranking,...,fl_4g,dt_ano_stts,idade,avg_msisdn,avg_cpf,trocou_aparelho,trocou_aparelho_previsto,aging_aparelho,aging_aparelho_previsto,dt_fim_previsto
0,5588988039599,724118596208177,35946008,2021-05-22,2022-08-06,1,APPLE,IPHONE 7 (A1660),359460083674180,1,...,1,2017,39,14.0,14.0,1,1,14,14,2022-07
1,5598984843818,724119893014054,35301109,2021-07-11,2022-09-15,1,APPLE,IPHONE 8 PLUS (A1864),353011090395470,1,...,1,2017,39,14.0,14.0,1,1,14,14,2022-09
2,5543991172181,724064190886669,35923309,2021-08-23,2022-09-10,1,SAMSUNG,SM-J810M DS,359233092097610,1,...,1,2018,44,12.0,12.0,1,1,12,12,2022-08
3,5551999774680,724065191700977,35616011,2021-04-14,2022-08-20,1,Samsung Korea,GALAXY A31,356160114319960,1,...,1,2020,43,16.0,16.0,1,1,16,16,2022-08
4,5565996867917,724066613532716,86830805,2021-07-06,2022-08-04,1,XIAOMI,REDMI 9A (M2006C3LG),868308057222690,1,...,1,2020,67,12.0,12.0,1,1,12,12,2022-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003070,5527997164405,724113002524393,35687011,2021-10-20,2022-11-14,1,APPLE,IPHONE 11 (A2111),356870110599300,1,...,1,2020,66,12.0,12.0,1,1,12,12,2022-10
1003071,5551996583605,724065010778518,35715510,2021-02-12,2022-07-31,1,LG Electronics Inc.,LM-X210BMW,357155106284230,1,...,1,2019,43,17.0,17.0,1,1,17,17,2022-07
1003072,5527998979742,724112706384504,35411310,2021-02-17,2022-07-26,1,Multilaser Industrial S.A.,VITA 3G,354113101160480,1,...,1,2018,54,17.0,17.0,1,1,17,17,2022-07
1003073,5554996750264,724065191279991,35556211,2021-04-05,2022-08-26,1,MOTOROLA,FIJI (XT2053-2),355562113384250,1,...,1,2020,39,16.0,16.0,1,1,16,16,2022-08


***
### Tabela Verdade

#### Comparando a tabela verdade de 'trocou_aparelho_previsto' com 'trocou_aparelho' para as classes

In [41]:
dados['ta_VP_ml'] = dados.apply(lambda x: '1' if x['trocou_aparelho'] == 1 and x['trocou_aparelho_previsto'] == 1
                                                else '0', axis=1)
dados['ta_FP_ml'] = dados.apply(lambda x: '1' if x['trocou_aparelho'] == 0 and x['trocou_aparelho_previsto'] == 1
                                                else '0', axis=1)
dados['ta_VN_ml'] = dados.apply(lambda x: '1' if x['trocou_aparelho'] == 0 and x['trocou_aparelho_previsto'] == 0
                                                else '0', axis=1)
dados['ta_FN_ml'] = dados.apply(lambda x: '1' if x['trocou_aparelho'] == 1 and x['trocou_aparelho_previsto'] == 0
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['ta_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['ta_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['ta_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['ta_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['ta_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['ta_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['ta_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['ta_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_ta_ml = ((len(dados[dados['ta_VP_ml'] == '1'])) / (len(dados[dados['ta_VP_ml'] == '1']) + len(dados[dados['ta_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_ta_ml * 100) + '%')

precision_ta_ml = ((len(dados[dados['ta_VP_ml'] == '1'])) / (len(dados[dados['ta_VP_ml'] == '1']) + len(dados[dados['ta_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_ta_ml * 100) + '%')
print()

f1_score_ta_ml = (2*precision_ta_ml*recall_ta_ml) / (precision_ta_ml + recall_ta_ml)
print('F1-Score %.2f' %float(f1_score_ta_ml * 100) + '%')
print()

accuracy_ta_ml = ((len(dados[dados['ta_VP_ml'] == '1'])) + (len(dados[dados['ta_VN_ml'] == '1']))) / \
((len(dados[dados['ta_VP_ml'] == '1'])) + (len(dados[dados['ta_VN_ml'] == '1'])) + (len(dados[dados['ta_FP_ml'] == '1'])) + (len(dados[dados['ta_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_ta_ml * 100) + '%')

Houve 908178 Acertos, ou 90.54% para VP
Houve 94897 Acertos, ou 9.46% para FP
Houve 0 Acertos, ou 0.00% para VN
Houve 0 Acertos, ou 0.00% para FN

Recall 100.00%
Precision 90.54%

F1-Score 95.03%

Accuracy 90.54%


#### Comparando a tabela verdade de 'avg_msisdn', 'avg_cpf' e 'previsto' com 'aging_aparelho' para as classes

#### 12

In [42]:
dados['12_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 12 and x['aging_aparelho_previsto'] == 12
                                                else '0', axis=1)
dados['12_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 12 and x['aging_aparelho_previsto'] == 12
                                                else '0', axis=1)
dados['12_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 12 and x['aging_aparelho_previsto'] != 12
                                                else '0', axis=1)
dados['12_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 12 and x['aging_aparelho_previsto'] != 12
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['12_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['12_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['12_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['12_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_12_ml = ((len(dados[dados['12_VP_ml'] == '1'])) / (len(dados[dados['12_VP_ml'] == '1']) + len(dados[dados['12_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_12_ml * 100) + '%')

precision_12_ml = ((len(dados[dados['12_VP_ml'] == '1'])) / (len(dados[dados['12_VP_ml'] == '1']) + len(dados[dados['12_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_12_ml * 100) + '%')
print()

f1_score_12_ml = (2*precision_12_ml*recall_12_ml) / (precision_12_ml + recall_12_ml)
print('F1-Score %.2f' %float(f1_score_12_ml * 100) + '%')
print()

accuracy_12_ml = ((len(dados[dados['12_VP_ml'] == '1'])) + (len(dados[dados['12_VN_ml'] == '1']))) / \
((len(dados[dados['12_VP_ml'] == '1'])) + (len(dados[dados['12_VN_ml'] == '1'])) + (len(dados[dados['12_FP_ml'] == '1'])) + (len(dados[dados['12_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_12_ml * 100) + '%')

Houve 159334 Acertos, ou 15.88% para VP
Houve 8051 Acertos, ou 0.80% para FP
Houve 812389 Acertos, ou 80.99% para VN
Houve 23301 Acertos, ou 2.32% para FN

Recall 87.24%
Precision 95.19%

F1-Score 91.04%

Accuracy 96.87%


In [43]:
dados['12_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 12 and x['avg_msisdn'] == 12
                                                else '0', axis=1)
dados['12_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 12 and x['avg_msisdn'] == 12
                                                else '0', axis=1)
dados['12_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 12 and x['avg_msisdn'] != 12
                                                else '0', axis=1)
dados['12_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 12 and x['avg_msisdn'] != 12
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['12_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['12_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['12_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['12_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_12_avg_msisdn = ((len(dados[dados['12_VP_avg_msisdn'] == '1'])) / (len(dados[dados['12_VP_avg_msisdn'] == '1']) + len(dados[dados['12_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_12_avg_msisdn * 100) + '%')

precision_12_avg_msisdn = ((len(dados[dados['12_VP_avg_msisdn'] == '1'])) / (len(dados[dados['12_VP_avg_msisdn'] == '1']) + len(dados[dados['12_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_12_avg_msisdn * 100) + '%')
print()

f1_score_12_avg_msisdn = (2*precision_12_avg_msisdn*recall_12_avg_msisdn) / (precision_12_avg_msisdn + recall_12_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_12_avg_msisdn * 100) + '%')
print()

accuracy_12_avg_msisdn = ((len(dados[dados['12_VP_avg_msisdn'] == '1'])) + (len(dados[dados['12_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['12_VP_avg_msisdn'] == '1'])) + (len(dados[dados['12_VN_avg_msisdn'] == '1'])) + (len(dados[dados['12_FP_avg_msisdn'] == '1'])) + (len(dados[dados['12_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_12_avg_msisdn * 100) + '%')

Houve 131066 Acertos, ou 13.07% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 820440 Acertos, ou 81.79% para VN
Houve 51569 Acertos, ou 5.14% para FN

Recall 71.76%
Precision 100.00%

F1-Score 83.56%

Accuracy 94.86%


In [44]:
dados['12_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 12 and x['avg_cpf'] == 12
                                                else '0', axis=1)
dados['12_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 12 and x['avg_cpf'] == 12
                                                else '0', axis=1)
dados['12_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 12 and x['avg_cpf'] != 12
                                                else '0', axis=1)
dados['12_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 12 and x['avg_cpf'] != 12
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['12_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['12_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['12_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['12_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['12_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_12_avg_cpf = ((len(dados[dados['12_VP_avg_cpf'] == '1'])) / (len(dados[dados['12_VP_avg_cpf'] == '1']) + len(dados[dados['12_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_12_avg_cpf * 100) + '%')

precision_12_avg_cpf = ((len(dados[dados['12_VP_avg_cpf'] == '1'])) / (len(dados[dados['12_VP_avg_cpf'] == '1']) + len(dados[dados['12_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_12_avg_cpf * 100) + '%')
print()

f1_score_12_avg_cpf = (2*precision_12_avg_cpf*recall_12_avg_cpf) / (precision_12_avg_cpf + recall_12_avg_cpf)
print('F1-Score %.2f' %float(f1_score_12_avg_cpf * 100) + '%')
print()

accuracy_12_avg_cpf = ((len(dados[dados['12_VP_avg_cpf'] == '1'])) + (len(dados[dados['12_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['12_VP_avg_cpf'] == '1'])) + (len(dados[dados['12_VN_avg_cpf'] == '1'])) + (len(dados[dados['12_FP_avg_cpf'] == '1'])) + (len(dados[dados['12_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_12_avg_cpf * 100) + '%')

Houve 116340 Acertos, ou 11.60% para VP
Houve 4911 Acertos, ou 0.49% para FP
Houve 815529 Acertos, ou 81.30% para VN
Houve 66295 Acertos, ou 6.61% para FN

Recall 63.70%
Precision 95.95%

F1-Score 76.57%

Accuracy 92.90%


#### 13

In [45]:
dados['13_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 13 and x['aging_aparelho_previsto'] == 13
                                                else '0', axis=1)
dados['13_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 13 and x['aging_aparelho_previsto'] == 13
                                                else '0', axis=1)
dados['13_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 13 and x['aging_aparelho_previsto'] != 13
                                                else '0', axis=1)
dados['13_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 13 and x['aging_aparelho_previsto'] != 13
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['13_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['13_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['13_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['13_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_13_ml = ((len(dados[dados['13_VP_ml'] == '1'])) / (len(dados[dados['13_VP_ml'] == '1']) + len(dados[dados['13_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_13_ml * 100) + '%')

precision_13_ml = ((len(dados[dados['13_VP_ml'] == '1'])) / (len(dados[dados['13_VP_ml'] == '1']) + len(dados[dados['13_FP_ml'] == '1'])))
print('Recall %.2f' %float(precision_13_ml * 100) + '%')
print()

f1_score_13_ml = (2*precision_13_ml*recall_13_ml) / (precision_13_ml + recall_13_ml)
print('F1-Score %.2f' %float(f1_score_13_ml * 100) + '%')
print()

accuracy_13_ml = ((len(dados[dados['13_VP_ml'] == '1'])) + (len(dados[dados['13_VN_ml'] == '1']))) / \
((len(dados[dados['13_VP_ml'] == '1'])) + (len(dados[dados['13_VN_ml'] == '1'])) + (len(dados[dados['13_FP_ml'] == '1'])) + (len(dados[dados['13_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_13_ml * 100) + '%')

Houve 140248 Acertos, ou 13.98% para VP
Houve 14371 Acertos, ou 1.43% para FP
Houve 825443 Acertos, ou 82.29% para VN
Houve 23013 Acertos, ou 2.29% para FN

Recall 85.90%
Recall 90.71%

F1-Score 88.24%

Accuracy 96.27%


In [46]:
dados['13_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 13 and x['avg_msisdn'] == 13
                                                else '0', axis=1)
dados['13_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 13 and x['avg_msisdn'] == 13
                                                else '0', axis=1)
dados['13_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 13 and x['avg_msisdn'] != 13
                                                else '0', axis=1)
dados['13_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 13 and x['avg_msisdn'] != 13
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['13_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['13_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['13_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['13_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_13_avg_msisdn = ((len(dados[dados['13_VP_avg_msisdn'] == '1'])) / (len(dados[dados['13_VP_avg_msisdn'] == '1']) + len(dados[dados['13_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_13_avg_msisdn * 100) + '%')

precision_13_avg_msisdn = ((len(dados[dados['13_VP_avg_msisdn'] == '1'])) / (len(dados[dados['13_VP_avg_msisdn'] == '1']) + len(dados[dados['13_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_13_avg_msisdn * 100) + '%')
print()

f1_score_13_avg_msisdn = (2*precision_13_avg_msisdn*recall_13_avg_msisdn) / (precision_13_avg_msisdn + recall_13_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_13_avg_msisdn * 100) + '%')
print()

accuracy_13_avg_msisdn = ((len(dados[dados['13_VP_avg_msisdn'] == '1'])) + (len(dados[dados['13_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['13_VP_avg_msisdn'] == '1'])) + (len(dados[dados['13_VN_avg_msisdn'] == '1'])) + (len(dados[dados['13_FP_avg_msisdn'] == '1'])) + (len(dados[dados['13_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_13_avg_msisdn * 100) + '%')

Houve 116875 Acertos, ou 11.65% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 839814 Acertos, ou 83.72% para VN
Houve 46386 Acertos, ou 4.62% para FN

Recall 71.59%
Precision 100.00%

F1-Score 83.44%

Accuracy 95.38%


In [47]:
dados['13_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 13 and x['avg_cpf'] == 13
                                                else '0', axis=1)
dados['13_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 13 and x['avg_cpf'] == 13
                                                else '0', axis=1)
dados['13_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 13 and x['avg_cpf'] != 13
                                                else '0', axis=1)
dados['13_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 13 and x['avg_cpf'] != 13
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['13_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['13_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['13_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['13_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['13_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_13_avg_cpf = ((len(dados[dados['13_VP_avg_cpf'] == '1'])) / (len(dados[dados['13_VP_avg_cpf'] == '1']) + len(dados[dados['13_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_13_avg_cpf * 100) + '%')

precision_13_avg_cpf = ((len(dados[dados['13_VP_avg_cpf'] == '1'])) / (len(dados[dados['13_VP_avg_cpf'] == '1']) + len(dados[dados['13_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_13_avg_cpf * 100) + '%')
print()

f1_score_13_avg_cpf = (2*precision_13_avg_cpf*recall_13_avg_cpf) / (precision_13_avg_cpf + recall_13_avg_cpf)
print('F1-Score %.2f' %float(f1_score_13_avg_cpf * 100) + '%')
print()

accuracy_13_avg_cpf = ((len(dados[dados['13_VP_avg_cpf'] == '1'])) + (len(dados[dados['13_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['13_VP_avg_cpf'] == '1'])) + (len(dados[dados['13_VN_avg_cpf'] == '1'])) + (len(dados[dados['13_FP_avg_cpf'] == '1'])) + (len(dados[dados['13_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_13_avg_cpf * 100) + '%')

Houve 103849 Acertos, ou 10.35% para VP
Houve 4157 Acertos, ou 0.41% para FP
Houve 835657 Acertos, ou 83.31% para VN
Houve 59412 Acertos, ou 5.92% para FN

Recall 63.61%
Precision 96.15%

F1-Score 76.57%

Accuracy 93.66%


#### 14

In [48]:
dados['14_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 14 and x['aging_aparelho_previsto'] == 14
                                                else '0', axis=1)
dados['14_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 14 and x['aging_aparelho_previsto'] == 14
                                                else '0', axis=1)
dados['14_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 14 and x['aging_aparelho_previsto'] != 14
                                                else '0', axis=1)
dados['14_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 14 and x['aging_aparelho_previsto'] != 14
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['14_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['14_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['14_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['14_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_14_ml = ((len(dados[dados['14_VP_ml'] == '1'])) / (len(dados[dados['14_VP_ml'] == '1']) + len(dados[dados['14_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_14_ml * 100) + '%')

precision_14_ml = ((len(dados[dados['14_VP_ml'] == '1'])) / (len(dados[dados['14_VP_ml'] == '1']) + len(dados[dados['14_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_14_ml * 100) + '%')
print()

f1_score_14_ml = (2*precision_14_ml*recall_14_ml) / (precision_14_ml + recall_14_ml)
print('F1-Score %.2f' %float(f1_score_14_ml * 100) + '%')
print()

accuracy_14_ml = ((len(dados[dados['14_VP_ml'] == '1'])) + (len(dados[dados['14_VN_ml'] == '1']))) / \
((len(dados[dados['14_VP_ml'] == '1'])) + (len(dados[dados['14_VN_ml'] == '1'])) + (len(dados[dados['14_FP_ml'] == '1'])) + (len(dados[dados['14_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_14_ml * 100) + '%')

Houve 130767 Acertos, ou 13.04% para VP
Houve 20017 Acertos, ou 2.00% para FP
Houve 832787 Acertos, ou 83.02% para VN
Houve 19504 Acertos, ou 1.94% para FN

Recall 87.02%
Precision 86.72%

F1-Score 86.87%

Accuracy 96.06%


In [49]:
dados['14_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 14 and x['avg_msisdn'] == 14
                                                else '0', axis=1)
dados['14_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 14 and x['avg_msisdn'] == 14
                                                else '0', axis=1)
dados['14_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 14 and x['avg_msisdn'] != 14
                                                else '0', axis=1)
dados['14_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 14 and x['avg_msisdn'] != 14
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['14_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['14_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['14_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['14_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_14_avg_msisdn = ((len(dados[dados['14_VP_avg_msisdn'] == '1'])) / (len(dados[dados['14_VP_avg_msisdn'] == '1']) + len(dados[dados['14_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_14_avg_msisdn * 100) + '%')

precision_14_avg_msisdn = ((len(dados[dados['14_VP_avg_msisdn'] == '1'])) / (len(dados[dados['14_VP_avg_msisdn'] == '1']) + len(dados[dados['14_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_14_avg_msisdn * 100) + '%')
print()

f1_score_14_avg_msisdn = (2*precision_14_avg_msisdn*recall_14_avg_msisdn) / (precision_14_avg_msisdn + recall_14_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_14_avg_msisdn * 100) + '%')
print()

accuracy_14_avg_msisdn = ((len(dados[dados['14_VP_avg_msisdn'] == '1'])) + (len(dados[dados['14_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['14_VP_avg_msisdn'] == '1'])) + (len(dados[dados['14_VN_avg_msisdn'] == '1'])) + (len(dados[dados['14_FP_avg_msisdn'] == '1'])) + (len(dados[dados['14_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_14_avg_msisdn * 100) + '%')

Houve 109163 Acertos, ou 10.88% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 852804 Acertos, ou 85.02% para VN
Houve 41108 Acertos, ou 4.10% para FN

Recall 72.64%
Precision 100.00%

F1-Score 84.15%

Accuracy 95.90%


In [50]:
dados['14_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 14 and x['avg_cpf'] == 14
                                                else '0', axis=1)
dados['14_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 14 and x['avg_cpf'] == 14
                                                else '0', axis=1)
dados['14_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 14 and x['avg_cpf'] != 14
                                                else '0', axis=1)
dados['14_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 14 and x['avg_cpf'] != 14
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['14_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['14_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['14_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['14_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['14_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_14_avg_cpf = ((len(dados[dados['14_VP_avg_cpf'] == '1'])) / (len(dados[dados['14_VP_avg_cpf'] == '1']) + len(dados[dados['14_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_14_avg_cpf * 100) + '%')

precision_14_avg_cpf = ((len(dados[dados['14_VP_avg_cpf'] == '1'])) / (len(dados[dados['14_VP_avg_cpf'] == '1']) + len(dados[dados['14_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_14_avg_cpf * 100) + '%')
print()

f1_score_14_avg_cpf = (2*precision_14_avg_cpf*recall_14_avg_cpf) / (precision_14_avg_cpf + recall_14_avg_cpf)
print('F1-Score %.2f' %float(f1_score_14_avg_cpf * 100) + '%')
print()

accuracy_14_avg_cpf = ((len(dados[dados['14_VP_avg_cpf'] == '1'])) + (len(dados[dados['14_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['14_VP_avg_cpf'] == '1'])) + (len(dados[dados['14_VN_avg_cpf'] == '1'])) + (len(dados[dados['14_FP_avg_cpf'] == '1'])) + (len(dados[dados['14_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_14_avg_cpf * 100) + '%')

Houve 96374 Acertos, ou 9.61% para VP
Houve 4923 Acertos, ou 0.49% para FP
Houve 847881 Acertos, ou 84.53% para VN
Houve 53897 Acertos, ou 5.37% para FN

Recall 64.13%
Precision 95.14%

F1-Score 76.62%

Accuracy 94.14%


#### 15

In [51]:
dados['15_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 15 and x['aging_aparelho_previsto'] == 15
                                                else '0', axis=1)
dados['15_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 15 and x['aging_aparelho_previsto'] == 15
                                                else '0', axis=1)
dados['15_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 15 and x['aging_aparelho_previsto'] != 15
                                                else '0', axis=1)
dados['15_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 15 and x['aging_aparelho_previsto'] != 15
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['15_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['15_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['15_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['15_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_15_ml = ((len(dados[dados['15_VP_ml'] == '1'])) / (len(dados[dados['15_VP_ml'] == '1']) + len(dados[dados['15_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_15_ml * 100) + '%')

precision_15_ml = ((len(dados[dados['15_VP_ml'] == '1'])) / (len(dados[dados['15_VP_ml'] == '1']) + len(dados[dados['15_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_15_ml * 100) + '%')
print()

f1_score_15_ml = (2*precision_15_ml*recall_15_ml) / (precision_15_ml + recall_15_ml)
print('F1-Score %.2f' %float(f1_score_15_ml * 100) + '%')
print()

accuracy_15_ml = ((len(dados[dados['15_VP_ml'] == '1'])) + (len(dados[dados['15_VN_ml'] == '1']))) / \
((len(dados[dados['15_VP_ml'] == '1'])) + (len(dados[dados['15_VN_ml'] == '1'])) + (len(dados[dados['15_FP_ml'] == '1'])) + (len(dados[dados['15_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_15_ml * 100) + '%')

Houve 119051 Acertos, ou 11.87% para VP
Houve 20116 Acertos, ou 2.01% para FP
Houve 846762 Acertos, ou 84.42% para VN
Houve 17146 Acertos, ou 1.71% para FN

Recall 87.41%
Precision 85.55%

F1-Score 86.47%

Accuracy 96.29%


In [52]:
dados['15_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 15 and x['avg_msisdn'] == 15
                                                else '0', axis=1)
dados['15_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 15 and x['avg_msisdn'] == 15
                                                else '0', axis=1)
dados['15_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 15 and x['avg_msisdn'] != 15
                                                else '0', axis=1)
dados['15_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 15 and x['avg_msisdn'] != 15
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['15_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['15_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['15_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['15_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_15_avg_msisdn = ((len(dados[dados['15_VP_avg_msisdn'] == '1'])) / (len(dados[dados['15_VP_avg_msisdn'] == '1']) + len(dados[dados['15_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_15_avg_msisdn * 100) + '%')

precision_15_avg_msisdn = ((len(dados[dados['15_VP_avg_msisdn'] == '1'])) / (len(dados[dados['15_VP_avg_msisdn'] == '1']) + len(dados[dados['15_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_15_avg_msisdn * 100) + '%')
print()

f1_score_15_avg_msisdn = (2*precision_15_avg_msisdn*recall_15_avg_msisdn) / (precision_15_avg_msisdn + recall_15_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_15_avg_msisdn * 100) + '%')
print()

accuracy_15_avg_msisdn = ((len(dados[dados['15_VP_avg_msisdn'] == '1'])) + (len(dados[dados['15_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['15_VP_avg_msisdn'] == '1'])) + (len(dados[dados['15_VN_avg_msisdn'] == '1'])) + (len(dados[dados['15_FP_avg_msisdn'] == '1'])) + (len(dados[dados['15_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_15_avg_msisdn * 100) + '%')

Houve 102067 Acertos, ou 10.18% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 866878 Acertos, ou 86.42% para VN
Houve 34130 Acertos, ou 3.40% para FN

Recall 74.94%
Precision 100.00%

F1-Score 85.68%

Accuracy 96.60%


In [53]:
dados['15_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 15 and x['avg_cpf'] == 15
                                                else '0', axis=1)
dados['15_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 15 and x['avg_cpf'] == 15
                                                else '0', axis=1)
dados['15_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 15 and x['avg_cpf'] != 15
                                                else '0', axis=1)
dados['15_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 15 and x['avg_cpf'] != 15
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['15_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['15_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['15_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['15_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['15_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_15_avg_cpf = ((len(dados[dados['15_VP_avg_cpf'] == '1'])) / (len(dados[dados['15_VP_avg_cpf'] == '1']) + len(dados[dados['15_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_15_avg_cpf * 100) + '%')

precision_15_avg_cpf = ((len(dados[dados['15_VP_avg_cpf'] == '1'])) / (len(dados[dados['15_VP_avg_cpf'] == '1']) + len(dados[dados['15_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_15_avg_cpf * 100) + '%')
print()

f1_score_15_avg_cpf = (2*precision_15_avg_cpf*recall_15_avg_cpf) / (precision_15_avg_cpf + recall_15_avg_cpf)
print('F1-Score %.2f' %float(f1_score_15_avg_cpf * 100) + '%')
print()

accuracy_15_avg_cpf = ((len(dados[dados['15_VP_avg_cpf'] == '1'])) + (len(dados[dados['15_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['15_VP_avg_cpf'] == '1'])) + (len(dados[dados['15_VN_avg_cpf'] == '1'])) + (len(dados[dados['15_FP_avg_cpf'] == '1'])) + (len(dados[dados['15_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_15_avg_cpf * 100) + '%')

Houve 90016 Acertos, ou 8.97% para VP
Houve 6825 Acertos, ou 0.68% para FP
Houve 860053 Acertos, ou 85.74% para VN
Houve 46181 Acertos, ou 4.60% para FN

Recall 66.09%
Precision 92.95%

F1-Score 77.25%

Accuracy 94.72%


#### 16

In [54]:
dados['16_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 16 and x['aging_aparelho_previsto'] == 16
                                                else '0', axis=1)
dados['16_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 16 and x['aging_aparelho_previsto'] == 16
                                                else '0', axis=1)
dados['16_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 16 and x['aging_aparelho_previsto'] != 16
                                                else '0', axis=1)
dados['16_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 16 and x['aging_aparelho_previsto'] != 16
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['16_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['16_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['16_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['16_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_16_ml = ((len(dados[dados['16_VP_ml'] == '1'])) / (len(dados[dados['16_VP_ml'] == '1']) + len(dados[dados['16_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_16_ml * 100) + '%')

precision_16_ml = ((len(dados[dados['16_VP_ml'] == '1'])) / (len(dados[dados['16_VP_ml'] == '1']) + len(dados[dados['16_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_16_ml * 100) + '%')
print()

f1_score_16_ml = (2*precision_16_ml*recall_16_ml) / (precision_16_ml + recall_16_ml)
print('F1-Score %.2f' %float(f1_score_16_ml * 100) + '%')
print()

accuracy_16_ml = ((len(dados[dados['16_VP_ml'] == '1'])) + (len(dados[dados['16_VN_ml'] == '1']))) / \
((len(dados[dados['16_VP_ml'] == '1'])) + (len(dados[dados['16_VN_ml'] == '1'])) + (len(dados[dados['16_FP_ml'] == '1'])) + (len(dados[dados['16_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_16_ml * 100) + '%')

Houve 113707 Acertos, ou 11.34% para VP
Houve 22803 Acertos, ou 2.27% para FP
Houve 851541 Acertos, ou 84.89% para VN
Houve 15024 Acertos, ou 1.50% para FN

Recall 88.33%
Precision 83.30%

F1-Score 85.74%

Accuracy 96.23%


In [55]:
dados['16_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 16 and x['avg_msisdn'] == 16
                                                else '0', axis=1)
dados['16_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 16 and x['avg_msisdn'] == 16
                                                else '0', axis=1)
dados['16_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 16 and x['avg_msisdn'] != 16
                                                else '0', axis=1)
dados['16_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 16 and x['avg_msisdn'] != 16
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['16_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['16_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['16_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['16_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_16_avg_msisdn = ((len(dados[dados['16_VP_avg_msisdn'] == '1'])) / (len(dados[dados['16_VP_avg_msisdn'] == '1']) + len(dados[dados['16_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_16_avg_msisdn * 100) + '%')

precision_16_avg_msisdn = ((len(dados[dados['16_VP_avg_msisdn'] == '1'])) / (len(dados[dados['16_VP_avg_msisdn'] == '1']) + len(dados[dados['16_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_16_avg_msisdn * 100) + '%')
print()

f1_score_16_avg_msisdn = (2*precision_16_avg_msisdn*recall_16_avg_msisdn) / (precision_16_avg_msisdn + recall_16_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_16_avg_msisdn * 100) + '%')
print()

accuracy_16_avg_msisdn = ((len(dados[dados['16_VP_avg_msisdn'] == '1'])) + (len(dados[dados['16_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['16_VP_avg_msisdn'] == '1'])) + (len(dados[dados['16_VN_avg_msisdn'] == '1'])) + (len(dados[dados['16_FP_avg_msisdn'] == '1'])) + (len(dados[dados['16_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_16_avg_msisdn * 100) + '%')

Houve 95127 Acertos, ou 9.48% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 874344 Acertos, ou 87.17% para VN
Houve 33604 Acertos, ou 3.35% para FN

Recall 73.90%
Precision 100.00%

F1-Score 84.99%

Accuracy 96.65%


In [56]:
dados['16_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 16 and x['avg_cpf'] == 16
                                                else '0', axis=1)
dados['16_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 16 and x['avg_cpf'] == 16
                                                else '0', axis=1)
dados['16_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 16 and x['avg_cpf'] != 16
                                                else '0', axis=1)
dados['16_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 16 and x['avg_cpf'] != 16
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['16_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['16_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['16_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['16_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['16_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_16_avg_cpf = ((len(dados[dados['16_VP_avg_cpf'] == '1'])) / (len(dados[dados['16_VP_avg_cpf'] == '1']) + len(dados[dados['16_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_16_avg_cpf * 100) + '%')

precision_16_avg_cpf = ((len(dados[dados['16_VP_avg_cpf'] == '1'])) / (len(dados[dados['16_VP_avg_cpf'] == '1']) + len(dados[dados['16_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_16_avg_cpf * 100) + '%')
print()

f1_score_16_avg_cpf = (2*precision_16_avg_cpf*recall_16_avg_cpf) / (precision_16_avg_cpf + recall_16_avg_cpf)
print('F1-Score %.2f' %float(f1_score_16_avg_cpf * 100) + '%')
print()

accuracy_16_avg_cpf = ((len(dados[dados['16_VP_avg_cpf'] == '1'])) + (len(dados[dados['16_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['16_VP_avg_cpf'] == '1'])) + (len(dados[dados['16_VN_avg_cpf'] == '1'])) + (len(dados[dados['16_FP_avg_cpf'] == '1'])) + (len(dados[dados['16_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_16_avg_cpf * 100) + '%')

Houve 83847 Acertos, ou 8.36% para VP
Houve 6160 Acertos, ou 0.61% para FP
Houve 868184 Acertos, ou 86.55% para VN
Houve 44884 Acertos, ou 4.47% para FN

Recall 65.13%
Precision 93.16%

F1-Score 76.66%

Accuracy 94.91%


#### 17

In [57]:
dados['17_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 17 and x['aging_aparelho_previsto'] == 17
                                                else '0', axis=1)
dados['17_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 17 and x['aging_aparelho_previsto'] == 17
                                                else '0', axis=1)
dados['17_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 17 and x['aging_aparelho_previsto'] != 17
                                                else '0', axis=1)
dados['17_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 17 and x['aging_aparelho_previsto'] != 17
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['17_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['17_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['17_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['17_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_17_ml = ((len(dados[dados['17_VP_ml'] == '1'])) / (len(dados[dados['17_VP_ml'] == '1']) + len(dados[dados['17_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_17_ml * 100) + '%')

precision_17_ml = ((len(dados[dados['17_VP_ml'] == '1'])) / (len(dados[dados['17_VP_ml'] == '1']) + len(dados[dados['17_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_17_ml * 100) + '%')
print()

f1_score_17_ml = (2*precision_17_ml*recall_17_ml) / (precision_17_ml + recall_17_ml)
print('F1-Score %.2f' %float(f1_score_17_ml * 100) + '%')
print()

accuracy_17_ml = ((len(dados[dados['17_VP_ml'] == '1'])) + (len(dados[dados['17_VN_ml'] == '1']))) / \
((len(dados[dados['17_VP_ml'] == '1'])) + (len(dados[dados['17_VN_ml'] == '1'])) + (len(dados[dados['17_FP_ml'] == '1'])) + (len(dados[dados['17_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_17_ml * 100) + '%')

Houve 108490 Acertos, ou 10.82% para VP
Houve 19241 Acertos, ou 1.92% para FP
Houve 861377 Acertos, ou 85.87% para VN
Houve 13967 Acertos, ou 1.39% para FN

Recall 88.59%
Precision 84.94%

F1-Score 86.73%

Accuracy 96.69%


In [58]:
dados['17_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 17 and x['avg_msisdn'] == 17
                                                else '0', axis=1)
dados['17_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 17 and x['avg_msisdn'] == 17
                                                else '0', axis=1)
dados['17_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 17 and x['avg_msisdn'] != 17
                                                else '0', axis=1)
dados['17_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 17 and x['avg_msisdn'] != 17
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['17_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['17_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['17_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['17_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_17_avg_msisdn = ((len(dados[dados['17_VP_avg_msisdn'] == '1'])) / (len(dados[dados['17_VP_avg_msisdn'] == '1']) + len(dados[dados['17_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_17_avg_msisdn * 100) + '%')

precision_17_avg_msisdn = ((len(dados[dados['17_VP_avg_msisdn'] == '1'])) / (len(dados[dados['17_VP_avg_msisdn'] == '1']) + len(dados[dados['17_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_17_avg_msisdn * 100) + '%')
print()

f1_score_17_avg_msisdn = (2*precision_17_avg_msisdn*recall_17_avg_msisdn) / (precision_17_avg_msisdn + recall_17_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_17_avg_msisdn * 100) + '%')
print()

accuracy_17_avg_msisdn = ((len(dados[dados['17_VP_avg_msisdn'] == '1'])) + (len(dados[dados['17_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['17_VP_avg_msisdn'] == '1'])) + (len(dados[dados['17_VN_avg_msisdn'] == '1'])) + (len(dados[dados['17_FP_avg_msisdn'] == '1'])) + (len(dados[dados['17_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_17_avg_msisdn * 100) + '%')

Houve 91853 Acertos, ou 9.16% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 880618 Acertos, ou 87.79% para VN
Houve 30604 Acertos, ou 3.05% para FN

Recall 75.01%
Precision 100.00%

F1-Score 85.72%

Accuracy 96.95%


In [59]:
dados['17_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 17 and x['avg_cpf'] == 17
                                                else '0', axis=1)
dados['17_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 17 and x['avg_cpf'] == 17
                                                else '0', axis=1)
dados['17_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 17 and x['avg_cpf'] != 17
                                                else '0', axis=1)
dados['17_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 17 and x['avg_cpf'] != 17
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['17_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['17_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['17_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['17_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['17_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_17_avg_cpf = ((len(dados[dados['17_VP_avg_cpf'] == '1'])) / (len(dados[dados['17_VP_avg_cpf'] == '1']) + len(dados[dados['17_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_17_avg_cpf * 100) + '%')

precision_17_avg_cpf = ((len(dados[dados['17_VP_avg_cpf'] == '1'])) / (len(dados[dados['17_VP_avg_cpf'] == '1']) + len(dados[dados['17_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_17_avg_cpf * 100) + '%')
print()

f1_score_17_avg_cpf = (2*precision_17_avg_cpf*recall_17_avg_cpf) / (precision_17_avg_cpf + recall_17_avg_cpf)
print('F1-Score %.2f' %float(f1_score_17_avg_cpf * 100) + '%')
print()

accuracy_17_avg_cpf = ((len(dados[dados['17_VP_avg_cpf'] == '1'])) + (len(dados[dados['17_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['17_VP_avg_cpf'] == '1'])) + (len(dados[dados['17_VN_avg_cpf'] == '1'])) + (len(dados[dados['17_FP_avg_cpf'] == '1'])) + (len(dados[dados['17_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_17_avg_cpf * 100) + '%')

Houve 80733 Acertos, ou 8.05% para VP
Houve 4201 Acertos, ou 0.42% para FP
Houve 876417 Acertos, ou 87.37% para VN
Houve 41724 Acertos, ou 4.16% para FN

Recall 65.93%
Precision 95.05%

F1-Score 77.86%

Accuracy 95.42%


#### 18

In [60]:
dados['18_VP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 18 and x['aging_aparelho_previsto'] == 18
                                                else '0', axis=1)
dados['18_FP_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 18 and x['aging_aparelho_previsto'] == 18
                                                else '0', axis=1)
dados['18_VN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 18 and x['aging_aparelho_previsto'] != 18
                                                else '0', axis=1)
dados['18_FN_ml'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 18 and x['aging_aparelho_previsto'] != 18
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['18_VP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_VP_ml'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['18_FP_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_FP_ml'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['18_VN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_VN_ml'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['18_FN_ml'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_FN_ml'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_18_ml = ((len(dados[dados['18_VP_ml'] == '1'])) / (len(dados[dados['18_VP_ml'] == '1']) + len(dados[dados['18_FN_ml'] == '1'])))
print('Recall %.2f' %float(recall_18_ml * 100) + '%')

precision_18_ml = ((len(dados[dados['18_VP_ml'] == '1'])) / (len(dados[dados['18_VP_ml'] == '1']) + len(dados[dados['18_FP_ml'] == '1'])))
print('Precision %.2f' %float(precision_18_ml * 100) + '%')
print()

f1_score_18_ml = (2*precision_18_ml*recall_18_ml) / (precision_18_ml + recall_18_ml)
print('F1-Score %.2f' %float(f1_score_18_ml * 100) + '%')
print()

accuracy_18_ml = ((len(dados[dados['18_VP_ml'] == '1'])) + (len(dados[dados['18_VN_ml'] == '1']))) / \
((len(dados[dados['18_VP_ml'] == '1'])) + (len(dados[dados['18_VN_ml'] == '1'])) + (len(dados[dados['18_FP_ml'] == '1'])) + (len(dados[dados['18_FN_ml'] == '1'])))
print('Accuracy %.2f' %float(accuracy_18_ml * 100) + '%')

Houve 109901 Acertos, ou 10.96% para VP
Houve 16978 Acertos, ou 1.69% para FP
Houve 866574 Acertos, ou 86.39% para VN
Houve 9622 Acertos, ou 0.96% para FN

Recall 91.95%
Precision 86.62%

F1-Score 89.20%

Accuracy 97.35%


In [61]:
dados['18_VP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 18 and x['avg_msisdn'] == 18
                                                else '0', axis=1)
dados['18_FP_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 18 and x['avg_msisdn'] == 18
                                                else '0', axis=1)
dados['18_VN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 18 and x['avg_msisdn'] != 18
                                                else '0', axis=1)
dados['18_FN_avg_msisdn'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 18 and x['avg_msisdn'] != 18
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['18_VP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_VP_avg_msisdn'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['18_FP_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_FP_avg_msisdn'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['18_VN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_VN_avg_msisdn'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['18_FN_avg_msisdn'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_FN_avg_msisdn'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_18_avg_msisdn = ((len(dados[dados['18_VP_avg_msisdn'] == '1'])) / (len(dados[dados['18_VP_avg_msisdn'] == '1']) + len(dados[dados['18_FN_avg_msisdn'] == '1'])))
print('Recall %.2f' %float(recall_18_avg_msisdn * 100) + '%')

precision_18_avg_msisdn = ((len(dados[dados['18_VP_avg_msisdn'] == '1'])) / (len(dados[dados['18_VP_avg_msisdn'] == '1']) + len(dados[dados['18_FP_avg_msisdn'] == '1'])))
print('Precision %.2f' %float(precision_18_avg_msisdn * 100) + '%')
print()

f1_score_18_avg_msisdn = (2*precision_18_avg_msisdn*recall_18_avg_msisdn) / (precision_18_avg_msisdn + recall_18_avg_msisdn)
print('F1-Score %.2f' %float(f1_score_18_avg_msisdn * 100) + '%')
print()

accuracy_18_avg_msisdn = ((len(dados[dados['18_VP_avg_msisdn'] == '1'])) + (len(dados[dados['18_VN_avg_msisdn'] == '1']))) / \
((len(dados[dados['18_VP_avg_msisdn'] == '1'])) + (len(dados[dados['18_VN_avg_msisdn'] == '1'])) + (len(dados[dados['18_FP_avg_msisdn'] == '1'])) + (len(dados[dados['18_FN_avg_msisdn'] == '1'])))
print('Accuracy %.2f' %float(accuracy_18_avg_msisdn * 100) + '%')

Houve 87736 Acertos, ou 8.75% para VP
Houve 0 Acertos, ou 0.00% para FP
Houve 883552 Acertos, ou 88.08% para VN
Houve 31787 Acertos, ou 3.17% para FN

Recall 73.41%
Precision 100.00%

F1-Score 84.66%

Accuracy 96.83%


In [62]:
dados['18_VP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 18 and x['avg_cpf'] == 18
                                                else '0', axis=1)
dados['18_FP_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 18 and x['avg_cpf'] == 18
                                                else '0', axis=1)
dados['18_VN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] != 18 and x['avg_cpf'] != 18
                                                else '0', axis=1)
dados['18_FN_avg_cpf'] = dados.apply(lambda x: '1' if x['aging_aparelho'] == 18 and x['avg_cpf'] != 18
                                                else '0', axis=1)

print('Houve ' + str(len(dados[dados['18_VP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_VP_avg_cpf'] == '1'])/len(dados))*100) + '% para VP')

print('Houve ' + str(len(dados[dados['18_FP_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_FP_avg_cpf'] == '1'])/len(dados))*100) + '% para FP')

print('Houve ' + str(len(dados[dados['18_VN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_VN_avg_cpf'] == '1'])/len(dados))*100) + '% para VN')

print('Houve ' + str(len(dados[dados['18_FN_avg_cpf'] == '1'])) + ' Acertos, ou %.2f'\
             %float((len(dados[dados['18_FN_avg_cpf'] == '1'])/len(dados))*100) + '% para FN')

print()

recall_18_avg_cpf = ((len(dados[dados['18_VP_avg_cpf'] == '1'])) / (len(dados[dados['18_VP_avg_cpf'] == '1']) + len(dados[dados['18_FN_avg_cpf'] == '1'])))
print('Recall %.2f' %float(recall_18_avg_cpf * 100) + '%')

precision_18_avg_cpf = ((len(dados[dados['18_VP_avg_cpf'] == '1'])) / (len(dados[dados['18_VP_avg_cpf'] == '1']) + len(dados[dados['18_FP_avg_cpf'] == '1'])))
print('Precision %.2f' %float(precision_18_avg_cpf * 100) + '%')
print()

f1_score_18_avg_cpf = (2*precision_18_avg_cpf*recall_18_avg_cpf) / (precision_18_avg_cpf + recall_18_avg_cpf)
print('F1-Score %.2f' %float(f1_score_18_avg_cpf * 100) + '%')
print()

accuracy_18_avg_cpf = ((len(dados[dados['18_VP_avg_cpf'] == '1'])) + (len(dados[dados['18_VN_avg_cpf'] == '1']))) / \
((len(dados[dados['18_VP_avg_cpf'] == '1'])) + (len(dados[dados['18_VN_avg_cpf'] == '1'])) + (len(dados[dados['18_FP_avg_cpf'] == '1'])) + (len(dados[dados['18_FN_avg_cpf'] == '1'])))
print('Accuracy %.2f' %float(accuracy_18_avg_cpf * 100) + '%')

Houve 77027 Acertos, ou 7.68% para VP
Houve 1237 Acertos, ou 0.12% para FP
Houve 882315 Acertos, ou 87.96% para VN
Houve 42496 Acertos, ou 4.24% para FN

Recall 64.45%
Precision 98.42%

F1-Score 77.89%

Accuracy 95.64%


In [76]:
print(classification_report(dados['aging_aparelho'], np.asarray(dados['aging_aparelho_previsto'], dtype = 'int')))

              precision    recall  f1-score   support

          12       0.95      0.87      0.91    182635
          13       0.91      0.86      0.88    163261
          14       0.87      0.87      0.87    150271
          15       0.86      0.87      0.86    136197
          16       0.83      0.88      0.86    128731
          17       0.85      0.89      0.87    122457
          18       0.87      0.92      0.89    119523

    accuracy                           0.88   1003075
   macro avg       0.88      0.88      0.88   1003075
weighted avg       0.88      0.88      0.88   1003075



In [77]:
print(classification_report(dados['aging_aparelho'], np.asarray(dados['avg_msisdn'], dtype = 'int')))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       1.00      0.72      0.84    182635
          13       1.00      0.72      0.83    163261
          14       1.00      0.73      0.84    150271
          15       1.00      0.75      0.86    136197
          16       1.00      0.74      0.85    128731
          17       1.00    

In [78]:
print(classification_report(dados['aging_aparelho'], np.asarray(dados['avg_cpf'], dtype = 'int')))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.92      0.64      0.76    182635
          13       0.92      0.64      0.76    163261
          14       0.91      0.65      0.76    150271
          15       0.89      0.67      0.76    136197
          16       0.90      0.66      0.76    128731
          17       0.92    